In [1]:
import os
import sys
import requests
import json

import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from tqdm import tqdm

AXIS_FONT_SIZE = 16

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# print(notebook_dir)
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

from data_processing import DataProcessing
from data_acquisition import OpenMeasuresDirector
from text_generation_models import TextGenerationModelFactory

In [2]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Configuring parameters
terms = "(nba OR mlb OR nfl) AND (will win)"
terms = "will win"
terms_for_query = OpenMeasuresDirector
limit = 10000
# Use default since and until: https://api.smat-app.com/docs#/default/content_content_get
since = '2024-01-01'
until = '2025-02-18' 
esquery = 'query_string' # Elasticsearch across all fields

sites = ["tiktok_comment", "bluesky", "truth_social"]
# sites = ["truth_social"]
hits_per_site_dfs = []
for site in sites:
    hits_for_site_df = OpenMeasuresDirector.construct_from_dataset(terms=terms, limit=limit, site=site, start_date=since, end_date=until, querytype=esquery)
    hits_per_site_dfs.append(hits_for_site_df)

Query's URL: http://api.smat-app.com/content?term=will win&limit=10&site=tiktok_comment&since=2024-01-01&until=2025-02-18&querytype=query_string
Query's URL: http://api.smat-app.com/content?term=will win&limit=10&site=bluesky&since=2024-01-01&until=2025-02-18&querytype=query_string
Query's URL: http://api.smat-app.com/content?term=will win&limit=10&site=truth_social&since=2024-01-01&until=2025-02-18&querytype=query_string


In [4]:
hits_per_site_dfs

[               author            author_id  author_pin             aweme_id  \
 0      fern_thatsbann  7317351016256881710       False  7299571998177611054   
 1            teeman01  6779046330004521990       False  7318494875060833579   
 2        topmarek2025  7213082520992056325       False  7318729394392976673   
 3        chymistgamer  7279908273892180997       False  7289124253989915946   
 4  inthecloverkingdom  7024639081057829893       False  7314701659458882859   
 5           gxoxolove  6616367097076170757       False  7313345818457656618   
 6         kingsford22  6731519913712747525       False  7318601332582206766   
 7      lonelymouse678  7019540473409291270       False  7317757260409605422   
 8           deproiski  6913957208511464454       False  7318048835593309486   
 9    mischievousamory  6812027599621391365       False  7318014126284999938   
 
                    cid  collect_stat          collected_by comment_language  \
 0  7318913038579843886             0 

- in json i'm saving collect more metadataa - use all cols given from each site
1. time for each query using query process with Dr. Grant


In [17]:
tiktok_df = hits_per_site_dfs[0]
bluesky_df = hits_per_site_dfs[1]
true_social_df = hits_per_site_dfs[2]
true_social_df['text'] = true_social_df['content_cleaned']
true_social_df.head(3)

,account,bookmarked,card,collected_by,content,content_cleaned,created_at,datatype,emojis,favourited,favourites_count,group,id,in_reply_to,in_reply_to_account_id,in_reply_to_id,language,media_attachments,mentions,muted,pinned,poll,quote,quote_id,reblog,reblogged,reblogs_count,replies_count,sensitive,spoiler_text,sponsored,tags,uri,url,visibility,lastseents,application,edited_at,text,tombstone,tv,version,downvotes_count,reaction,upvotes_count,votable,editable,Site
0,"{'acct': 'monkeybut', 'display_name': 'monkeybut', 'id': '108399588319928832', 'username': 'monkeybut'}",False,None,smat-scrapy-crawlers,"<p>RT <span class=""h-card""><a href=""https://truthsocial.com/@Kash"" class=""u-url mention"">@<span>Kash</span></a></span>I did many trips with this fake news queen and 45, she always glowingly praised him. But when the circus came calling, she followed the clown show for her pay day. The difference- we will always fight for her right to say dumb shi! Keep printing it… we will save democracy behind <span class=""h-card""><a href=""https://truthsocial.com/@realDonaldTrump"" class=""u-url mention"">@<span>realDonaldTrump</span></a></span></p>","RT @KashI did many trips with this fake news queen and 45, she always glowingly praised him. But when the circus came calling, she followed the clown show for her pay day. The difference- we will always fight for her right to say dumb shi! Keep printing it… we will save democracy behind @realDonaldTrump",2024-01-01T00:00:03.605+00:00,post,[],False,0,None,111677748255410789,None,None,None,None,[],[],False,False,None,None,None,"{'id': '111677535128303666', 'uri': 'https://truthsocial.com/@Kash/111677535128303666', 'url': 'https://truthsocial.com/@Kash/111677535128303666'}",False,0,0,False,,False,[],https://truthsocial.com/@monkeybut/111677748255410789,https://truthsocial.com/@monkeybut/111677748255410789,public,2024-03-04T18:45:50.859991,NaN,NaN,"RT @KashI did many trips with this fake news queen and 45, she always glowingly praised him. But when the circus came calling, she followed the clown show for her pay day. The difference- we will always fight for her right to say dumb shi! Keep printing it… we will save democracy behind @realDonaldTrump",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,truth_social
1,"{'acct': 'Steveusmc88', 'display_name': 'Steveusmc88', 'id': '109264572624958124', 'username': 'Steveusmc88'}",False,None,smat-scrapy-crawlers,"<p>RT <span class=""h-card""><a href=""https://truthsocial.com/@TheGreatAmericanPodcast"" class=""u-url mention"">@<span>TheGreatAmericanPodcast</span></a></span>Good Morning Family we will be live today at 2pm cst come join us on <a href=""https://rumble.com/c/TheGreatAmericanPodcast"" rel=""nofollow noopener noreferrer"" target=""_blank""><span class=""invisible"">https://</span><span class=""ellipsis"">rumble.com/c/TheGreatAmericanP</span><span class=""invisible"">odcast</span></a></p>",RT @TheGreatAmericanPodcastGood Morning Family we will be live today at 2pm cst come join us on https://rumble.com/c/TheGreatAmericanPodcast,2024-01-01T00:00:04.878+00:00,post,[],False,0,"{'avatar': 'https://static-assets-1.truthsocial.com/tmtg:prime-ts-assets/groups/avatars/110/408/591/063/939/677/original/3608f556cfa3e959.png', 'avatar_static': 'https://static-assets-1.truthsocial.com/tmtg:prime-ts-assets/groups/avatars/110/408/591/063/939/677/original/3608f556cfa3e959.png', 'created_at': '2023-05-21T20:36:41.000Z', 'discoverable': True, 'display_name': 'Forever Thread Fam and Frens', 'group_visibility': 'everyone', 'header': 'https://static-assets-1.truthsocial.com/tmtg:prime-ts-assets/groups/headers/110/408/591/063/939/677/original/abc16e32be3b1792.png', 'header_static': 'https://static-assets-1.truthsocial.com/tmtg:prime-ts-assets/groups/headers/110/408/591/063/939/677/original/abc16e32be3b1792.png', 'id': '110408591063939677', 'locked': False, 'members_count': 303...",111677748338988853,None,None,None,None,[],[],False,False,None,None,None,"{'id': '111676166785110679', 'u

In [6]:
# true_social_df['text'] == true_social_df['content_cleaned']

In [7]:
tgmf = TextGenerationModelFactory()

# Groq Cloud (https://console.groq.com/docs/overview)
gemma_29b_generation_model = tgmf.create_instance('gemma2-9b-it') 
llama_318b_instant_generation_model = tgmf.create_instance('llama-3.1-8b-instant') 
llama_3370b_versatile_generation_model = tgmf.create_instance('llama-3.3-70b-versatile')  
llama_guard_4_12b_generation_model = tgmf.create_instance('meta-llama/llama-guard-4-12b')  

models = [gemma_29b_generation_model, llama_318b_instant_generation_model, llama_3370b_versatile_generation_model, llama_guard_4_12b_generation_model]

In [11]:
import json
import os

def save_to_json(data, path, site):
    site = f"{site}s"
    print(site)
    file_number = len([file for file in os.listdir(path) if file.startswith('{site}-')]) + 1
    file_name = f'{site}-{file_number}.json'
    file_path = os.path.join(path, file_name)
    with open(file_path, 'w') as f:
        json.dump(data, f)

In [12]:
# give our prediction defition for context

In [13]:
def detect_predictions_with_llms(df: pd.DataFrame, notebook_dir: str, site: str):
    labels = []
    batch_size = 50
    show_data = 1

    sentences = DataProcessing.df_to_list(df, col='text')
    
    for batch_idx in tqdm(range(0, len(sentences), batch_size)):
        batch = sentences[batch_idx:batch_idx+batch_size]
        for sentences_idx in tqdm(range(len(batch))):
            sentence = batch[sentences_idx]
            if show_data <=3: 
                print(f"       {sentence} --- {site}")
                show_data += 1
            prompt = f"Given this sentence ({sentence}), state if the sentence is a prediction, not a prediction, or not enough information. Also, if it is a prediction, state the prediction domain if any are finance, health, weather, policy, sports, or miscellaneous. Do not explain or provide any other details. Only state prediction, not a prediction, or not enough information."
            # print(f"Prompt: {prompt}")
            for model in models:  
                input_prompt = model.user(prompt)
                # print(input_prompt)  
                
                raw_text_llm_generation = model.chat_completion([input_prompt])
                # print(raw_text_llm_generation)
                # print("====================================")
                for line in raw_text_llm_generation.split("\n"):
                    # print(line)
                    if line.strip():
                        labels.append({"sentence": sentence, "model": model.__name__(), "label": line})
        save_dir = os.path.dirname(notebook_dir)
        save_dir = os.path.join(save_dir, 'data', 'open_measures', f"{site}s")
        save_to_json(labels, save_dir, site)
    return labels

In [14]:
tiktok_site = tiktok_df['Site'][0]
llms_generated_for_tiktok_comments = detect_predictions_with_llms(tiktok_df, notebook_dir, tiktok_site)
llms_generated_for_tiktok_comments_df = pd.DataFrame(llms_generated_for_tiktok_comments).rename(columns={'sentence': 'Text', 'model': 'Model', 'label': 'Label'})
llms_generated_for_tiktok_comments_df['Site'] = tiktok_site
llms_generated_for_tiktok_comments_df.head(7)

  0%|          | 0/1 [00:00<?, ?it/s]

       We will not be silenced. Stay strong <3 --- tiktok_comment


       Safest and quickest way to do it, by doing on the side will do damage your back --- tiktok_comment


       with usa gems package, it will be TH 14 in no time 🤙 --- tiktok_comment


100%|██████████| 1/1 [00:17<00:00, 17.07s/it]

tiktok_comments


,Text,Model,Label,Site
0,We will not be silenced. Stay strong <3,gemma2-9b-it,Not a prediction.,tiktok_comment
1,We will not be silenced. Stay strong <3,llama-3.1-8b-instant,Not a prediction,tiktok_comment
2,We will not be silenced. Stay strong <3,llama-3.3-70b-versatile,Not a prediction,tiktok_comment
3,We will not be silenced. Stay strong <3,meta-llama/llama-guard-4-12b,safe,tiktok_comment
4,"Safest and quickest way to do it, by doing on the side will do damage your back",gemma2-9b-it,Prediction,tiktok_comment
5,"Safest and quickest way to do it, by doing on the side will do damage your back",gemma2-9b-it,Health,tiktok_comment
6,"Safest and quickest way to do it, by doing on the side will do damage your back",llama-3.1-8b-instant,"Prediction, health.",tiktok_comment


In [ ]:
bluesky_site = bluesky_df['Site'][0]
llms_generated_for_bluesocial_comments = detect_predictions_with_llms(bluesky_df, notebook_dir, bluesky_site)
llms_generated_for_bluesocial_comments_df = pd.DataFrame(llms_generated_for_bluesocial_comments).rename(columns={'sentence': 'Text', 'model': 'Model', 'label': 'Label'})
llms_generated_for_bluesocial_comments_df.head(7)

  0%|          | 0/1 [00:00<?, ?it/s]

       The following thread contains the matches that are currently official for the Wildcats: Death and Defiance card, the exclusive Wildcats only mega event!

As always, the card is subject to change, and more matches will be announced following Wildcats 206. --- bluesky


       #K12 #SuicidePrevention #BehavioralHealthIntegration

https://stateofreform.com/featured/2021/06/michigan-schools-will-implement-behavioral-health-software-bh-works/?utm_campaign=meetedgar&utm_medium=social&utm_source=meetedgar.com

#SharedFacts --- bluesky


       my energy going into 2024 is "butter scraped over too much bread" and like Bilbo I will be looking into how I can go freeload off the elves until it's my turn to sail into the west --- bluesky


100%|██████████| 1/1 [00:14<00:00, 14.79s/it]

blueskys


,Text,Model,Label
0,"The following thread contains the matches that are currently official for the Wildcats: Death and Defiance card, the exclusive Wildcats only mega event!\n\nAs always, the card is subject to change, and more matches will be announced following Wildcats 206.",gemma2-9b-it,Not a prediction
1,"The following thread contains the matches that are currently official for the Wildcats: Death and Defiance card, the exclusive Wildcats only mega event!\n\nAs always, the card is subject to change, and more matches will be announced following Wildcats 206.",llama-3.1-8b-instant,"Prediction, sports."
2,"The following thread contains the matches that are currently official for the Wildcats: Death and Defiance card, the exclusive Wildcats only mega event!\n\nAs always, the card is subject to change, and more matches will be announced following Wildcats 206.",llama-3.3-70b-versatile,Not a prediction
3,"The following thread contains the matches that are currently official for the Wildcats: Death and Defiance card, the exclusive Wildcats only mega event!\n\nAs always, the card is subject to change, and more matches will be announced following Wildcats 206.",meta-llama/llama-guard-4-12b,unsafe
4,"The following thread contains the matches that are currently official for the Wildcats: Death and Defiance card, the exclusive Wildcats only mega event!\n\nAs always, the card is subject to change, and more matches will be announced following Wildcats 206.",meta-llama/llama-guard-4-12b,S8
5,#K12 #SuicidePrevention #BehavioralHealthIntegration\n\nhttps://stateofreform.com/featured/2021/06/michigan-schools-will-implement-behavioral-health-software-bh-works/?utm_campaign=meetedgar&utm_medium=social&utm_source=meetedgar.com\n\n#SharedFacts,gemma2-9b-it,Not a prediction
6,#K12 #SuicidePrevention #BehavioralHealthIntegration\n\nhttps://stateofreform.com/featured/2021/06/michigan-schools-will-implement-behavioral-health-software-bh-works/?utm_campaign=meetedgar&utm_medium=social&utm_source=meetedgar.com\n\n#SharedFacts,llama-3.1-8b-instant,"Prediction, policy."


In [21]:
true_social_site = true_social_df['Site'][0]
llms_generated_for_true_social_comments = detect_predictions_with_llms(true_social_df, notebook_dir, true_social_site)
llms_generated_for_true_social_comments_df = pd.DataFrame(llms_generated_for_true_social_comments).rename(columns={'sentence': 'Text', 'model': 'Model', 'label': 'Label'})
llms_generated_for_true_social_comments_df.head(7)

  0%|          | 0/1 [00:00<?, ?it/s]

       RT @KashI did many trips with this fake news queen and 45, she always glowingly praised him. But when the circus came calling, she followed the clown show for her pay day. The difference- we will always fight for her right to say dumb shi! Keep printing it… we will save democracy behind @realDonaldTrump --- truth_social


       RT @TheGreatAmericanPodcastGood Morning Family we will be live today at 2pm cst come join us on https://rumble.com/c/TheGreatAmericanPodcast --- truth_social


       God will not be mocked, watch out for a big bolder coming out of the sky. --- truth_social


100%|██████████| 1/1 [00:19<00:00, 19.24s/it]

truth_socials


,Text,Model,Label
0,"RT @KashI did many trips with this fake news queen and 45, she always glowingly praised him. But when the circus came calling, she followed the clown show for her pay day. The difference- we will always fight for her right to say dumb shi! Keep printing it… we will save democracy behind @realDonaldTrump",gemma2-9b-it,Not a prediction
1,"RT @KashI did many trips with this fake news queen and 45, she always glowingly praised him. But when the circus came calling, she followed the clown show for her pay day. The difference- we will always fight for her right to say dumb shi! Keep printing it… we will save democracy behind @realDonaldTrump",llama-3.1-8b-instant,Not a prediction.
2,"RT @KashI did many trips with this fake news queen and 45, she always glowingly praised him. But when the circus came calling, she followed the clown show for her pay day. The difference- we will always fight for her right to say dumb shi! Keep printing it… we will save democracy behind @realDonaldTrump",llama-3.3-70b-versatile,Not a prediction
3,"RT @KashI did many trips with this fake news queen and 45, she always glowingly praised him. But when the circus came calling, she followed the clown show for her pay day. The difference- we will always fight for her right to say dumb shi! Keep printing it… we will save democracy behind @realDonaldTrump",meta-llama/llama-guard-4-12b,unsafe
4,"RT @KashI did many trips with this fake news queen and 45, she always glowingly praised him. But when the circus came calling, she followed the clown show for her pay day. The difference- we will always fight for her right to say dumb shi! Keep printing it… we will save democracy behind @realDonaldTrump",meta-llama/llama-guard-4-12b,S5
5,RT @TheGreatAmericanPodcastGood Morning Family we will be live today at 2pm cst come join us on https://rumble.com/c/TheGreatAmericanPodcast,gemma2-9b-it,Not a prediction
6,RT @TheGreatAmericanPodcastGood Morning Family we will be live today at 2pm cst come join us on https://rumble.com/c/TheGreatAmericanPodcast,llama-3.1-8b-instant,Prediction (Miscellaneous)
